In [1]:
pip install -q folium

In [2]:
!pip install -q geopandas

In [3]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import json
import shapely
from shapely.geometry import shape, GeometryCollection, Point, Polygon
import geopandas as gpd

In [4]:
jeff_vlc = pd.read_csv('jeff_vlc_poi.csv', sep=',', decimal='.')
jeff_vlc.drop(['Unnamed: 0','Unnamed: 0.1','partner_id','prtn_address_latitude','prtn_address_longitude','order_id', 'Dist_Cliente_Tienda'],inplace=True,axis=1)

In [5]:
with open('seccensales.JSON') as f:
    seccensales_geojson = json.load(f)

In [6]:
def DrawLaundry_Points(latitude, longitude, map,string):
    folium.Marker(location=[latitude,longitude ],popup=string).add_to( map )

In [7]:
def get_seccensal(lat,long, infeatures):
  seccensal = 0
  point1 = Point(long, lat)
  for feature in infeatures["features"]:
      poly = Polygon(feature["geometry"]["coordinates"][0])
      if poly.contains(point1) == True:
          seccensal = feature["properties"]["coddistsec"]
          break
  return seccensal

In [8]:
def draw_map_cluster(dataset):
  map = folium.Map(location=[39.46994829189022, -0.37787440832473984], tiles='CartoDB dark_matter', zoom_start=14)
  marker_cluster = MarkerCluster().add_to(map)
  dataset_visual = dataset
  dataset_visual = dataset_visual.drop_duplicates('customer_id')
  dataset_visual.apply(lambda row: DrawLaundry_Points(row['ord_billing_address_lat'],row['ord_billing_address_long'],marker_cluster,row['prtn_code']), axis=1)
  return map

In [9]:
jeff_vlc["coddistsec"] = jeff_vlc.apply(lambda row: get_seccensal(row["ord_billing_address_lat"], row["ord_billing_address_long"], seccensales_geojson), axis=1) 
jeff_vlc = jeff_vlc[jeff_vlc["coddistsec"]!= 0]

Pedidos de las dos modalidades

In [ ]:
map_total = draw_map_cluster(jeff_vlc)
map_total

Solo pedidos offline

In [ ]:
jeff_vlc_offline=jeff_vlc[jeff_vlc['ord_channel_type_code']=='OFFLINE']
map_total_offline = draw_map_cluster(jeff_vlc_offline)
map_total_offline

In [ ]:
jeff_vlc_online=jeff_vlc[jeff_vlc['ord_channel_type_code']=='ONLINE']
map_total_online = draw_map_cluster(jeff_vlc_online)
map_total_online

**Tienda petxina**

Canal Offline Petxina

In [ ]:
jeff_vlc_offline_petxina=jeff_vlc_offline[jeff_vlc_offline['prtn_code']=='PETXINA']
map_total_offline_petxina = draw_map_cluster(jeff_vlc_offline_petxina)
map_total_offline_petxina

Canal Online Petxina

In [ ]:
jeff_vlc_online_petxina=jeff_vlc_online[jeff_vlc_online['prtn_code']=='PETXINA']
map_total_online_petxina = draw_map_cluster(jeff_vlc_online_petxina)
map_total_online_petxina

**Tienda Reino**

Canal Offline Reino

In [ ]:
jeff_vlc_offline_reino=jeff_vlc_offline[jeff_vlc_offline['prtn_code']=='REINO']
map_total_offline_reino = draw_map_cluster(jeff_vlc_offline_reino)
map_total_offline_reino

Canal Online Reino

In [ ]:
jeff_vlc_online_reino=jeff_vlc_online[jeff_vlc_online['prtn_code']=='REINO']
map_total_online_reino = draw_map_cluster(jeff_vlc_online_reino)
map_total_online_reino

**Correlaciones valencia**

In [17]:
def get_correlation_shop(in_vlc):
  customer_id_count = in_vlc[['customer_id','total_poi']]
  customer_id_count = customer_id_count.groupby(by=['customer_id']).count()
  customer_id_count=customer_id_count.reset_index(level='customer_id')
  customer_id_count = customer_id_count.rename(columns={'total_poi': 'num_operaciones'})
  result = pd.merge(customer_id_count, in_vlc, on="customer_id")
  result_final = result.drop_duplicates('customer_id')
  result_final.reset_index(inplace=True)
  result_final.drop(['customer_id','index','prtn_code','prtn_city_code','ord_channel_code','ord_channel_type_code','ord_logistics_code','ord_creation_date_time','coddistsec', 'ord_billing_address_lat', 'ord_billing_address_long'],inplace=True,axis=1)
  resultados_correlacion = result_final.corr()
  result_corr = resultados_correlacion['num_operaciones']
  return result_corr

Correlacion Petxina offline y online

In [18]:
result_petxina_online = get_correlation_shop(jeff_vlc_online_petxina)
result_petxina_offline = get_correlation_shop(jeff_vlc_offline_petxina)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
result_petxina_online

num_operaciones    1.000000
education          0.032288
lavanderias       -0.008336
tiendas           -0.019727
transporte        -0.075948
gimnasios         -0.073093
parques           -0.048690
restaurante        0.037888
bancos            -0.008972
estadios          -0.041878
hospitales         0.067472
total_poi         -0.015762
Name: num_operaciones, dtype: float64

In [20]:
result_petxina_offline

num_operaciones    1.000000
education         -0.188685
lavanderias       -0.188381
tiendas           -0.256605
transporte         0.390698
gimnasios         -0.206715
parques            0.392787
restaurante        0.015441
bancos             0.428307
estadios          -0.035762
hospitales        -0.325821
total_poi          0.126258
Name: num_operaciones, dtype: float64

Correlacion Reino offline y online

In [21]:
result_reino_online = get_correlation_shop(jeff_vlc_online_reino)
result_reino_offline = get_correlation_shop(jeff_vlc_offline_reino)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
result_reino_online

num_operaciones    1.000000
education         -0.037196
lavanderias        0.007891
tiendas            0.015815
transporte        -0.057881
gimnasios          0.088012
parques           -0.080456
restaurante        0.067028
bancos             0.011383
estadios          -0.019360
hospitales        -0.059537
total_poi          0.005779
Name: num_operaciones, dtype: float64

In [23]:
result_reino_offline

num_operaciones    1.000000
education         -0.372207
lavanderias       -0.395685
tiendas           -0.036360
transporte         0.390427
gimnasios         -0.215273
parques            0.393031
restaurante       -0.261033
bancos             0.362769
estadios           0.246702
hospitales         0.121555
total_poi         -0.066432
Name: num_operaciones, dtype: float64